# Day 8: Handheld Halting

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

## Puzzle 1

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

    acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
    jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
    nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:

```text
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

These instructions are visited in this order:

```text
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```

First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [1]:
# Python imports
from pathlib import Path
from typing import List, Tuple

Our solution here is to create a "machine" that runs the program. The machine works like a Turing machine. It has an accumulator that holds a value, a list of instructions as `(opcode, value)` tuples, and a pointer that holds the index of the current instruction in the program list.

Each opcode in the program gets its own *method* in the machine that adjusts the pointer and accumulator values according to the instructions in the program.

When it runs, the machine steps through the program, following the instructions in the program as defined above by calling the machine's *methods* with the values in the program's instruction.

As we need to test for infinite loops, we give the machine a history of which pointer values have been seen. An infinite loop would visit the same pointer twice so, when that happens, we raise an error.

We assume that attempting to point to a pointer position outside the program is a valid way to stop the program, and report the value in the accumulator at that point.

In [2]:
class Instruction:
    
    """Represents an instruction for the Machine"""
    
    opcode: str
    val: int
        
    def __init__(self, opcode: str, val: int) -> None:
        """Instantiate an instruction.
        
        :param opcode:  instruction name
        :param val:  instruction parameter
        """
        self.opcode = opcode
        self.val = val
        
    def __repr__(self) -> None:
        return f"Instruction <id(self)>: {opcode} {val}"

    def __str__(self) -> None:
        return f"{self.opcode} {self.val}"

    
class Machine:
    
    """Represents a machine for solving the puzzles"""
    
    acc: int = 0  # accumulator
    _history: List[int]  # history of visited pointer values
    prog: List  # the machine's program
    ptr: int = 0  # pointer to a line in the program
    verbose: False  # if True, prints the instructions as they are executed
        
    def __init__(self, verbose=False) -> None:
        """Instantiate machine."""
        self.reset()
                
        self.initialise_opcodes()
        
        self.verbose = verbose

    def _acc(self, val: int) -> None:
        """Perform acc instruction
        
        :param val:  increment for the accumulator
        """
        self.acc += val
        self.ptr += 1
        
    def _check_history(self) -> None:
        """Raise runtime error if current pointer value is in history"""
        if self.ptr in self._history:
            raise RuntimeError(f"Infinite Loop: {self.acc}")
        
    def _get_next_instruction(self) -> Instruction:
        """Return program instruction at current pointer"""
        opcode, val = self.prog[self.ptr]
        
        return Instruction(opcode, val)
    
    def initialise_opcodes(self) -> None:
        """Create distribution dictionary of methods for opcodes"""
        self.opcode_fn = {
            "acc": self._acc,
            "jmp": self._jmp,
            "nop": self._nop,
        }
    
    def _jmp(self, val: int) -> None:
        """Perform jmp instruction:
        
        :param val:  number of instructions to jump
        
        negative values jump backwards in the program; positive values
        jump forwards
        """
        self.ptr += val
    
    def load_program(self, fpath: str) -> None:
        """Load instructions from file"""
        with Path(fpath).open("r") as ifh:
            for line in ifh.readlines():
                ins, val = line.strip().split()
                self.prog.append((ins, int(val)))
    
    def _nop(self, val: int) -> None:
        """Perform nop instruction
        
        :param val:  no action is performed with this value
        """
        self.ptr += 1
    
    def read_program(self, prog: List[Tuple[str, int]]) -> None:
        """Read passed program
        
        :param prog:  machine program as a list of (instruction, value) tuples
        """
        self.reset_program()
        self.prog = prog

    def reset(self) -> None:
        """Clear machine state"""
        self.reset_accumulator()
        self.reset_history()
        self.reset_program()
        self.reset_pointer()
        
    def reset_accumulator(self) -> None:
        """Clear accumulator"""
        self.acc = 0
        
    def reset_history(self) -> None:
        """Clear pointer history"""
        self._history = []

    def reset_program(self) -> None:
        """Clear program"""
        self.prog = []

    def reset_pointer(self) -> None:
        """Clear pointer"""
        self.ptr = 0      
        
    def run(self) -> int:
        """Run current program"""
        while True:
            self._check_history()  # guard against infinite loops
            self._history.append(self.ptr)
            try:
                ins = self._get_next_instruction()
            except IndexError:  # program completes
                return self.acc
            if self.verbose:
                print(f"{self.ptr:04d}: {ins}")
            self._run_instruction(ins)
            
    def _run_instruction(self, ins: Instruction) -> None:
        """Execute passed Instruction
        
        :param ins:  Instruction instance to execute
        """
        self.opcode_fn[ins.opcode](ins.val)
        
    def __repr__(self):
        return f"Machine <id(self)>, {len(self.prog)} instructions"
    
    def __str__(self):
        return "\n".join([f"{opn} {val}" for opn, val in self.prog])

Solve the test puzzle.

In [3]:
machine = Machine()
machine.load_program("day08_test.txt")
try:
    machine.run()
except RuntimeError as err:
    print(err)

Infinite Loop: 5


Solve the real puzzle.

In [4]:
machine = Machine()
machine.load_program("day08_data.txt")
try:
    machine.run()
except RuntimeError as err:
    print(err)

Infinite Loop: 2080


## Part Two

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

```text
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

```text
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```

After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

Now we need to load the program and mutate/manipulate it, outside the machine. So we first write a function that loads a program for the machine into a variable.

In [5]:
def load_program(fpath: str) -> List[Tuple[str, int]]:
    """Load instructions from file:
    
    :param fpath:  path to program file
    """
    inst = []
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            ins, val = line.strip().split()
            inst.append((ins, int(val)))
    return inst

Then we write a function that takes a program as input, and returns a list containing that program, and all other programs where a pair of specified instructions are swapped exactly once.

In [6]:
def mutate_programs(prog: List[Tuple[str, int]], ins1: str, ins2: str) -> List[List[Tuple[str, int]]]:
    """Return all programs with exactly one ins1 and ins2 swapped
    
    :param prog:  base program to be mutated
    :param ins1:  instruction code that can be swapped
    :param ins2:  instruction code that can be swapped
    
    Returns all variants of the program with either...
    
    - one instance of ins1 replaced with ins2
    
    or
    
    - one instance of ins2 replaced with ins1
    """
    new_programs = [prog]
    for linenum, inst in enumerate(prog):
        if inst[0] == ins1:
            newprog = prog[:linenum] + [(ins2, inst[1])] + prog[linenum + 1:]
            new_programs.append(newprog)
        if inst[0] == ins2:
            newprog = prog[:linenum] + [(ins1, inst[1])] + prog[linenum + 1:]
            new_programs.append(newprog)
    return new_programs


Then we load our original program, generate the list of all mutated programs as specified in the puzzle, then loop over those programs one-by-one, giving them to the machine and running them.

In [7]:
inst = load_program("day08_test.txt")
programs = mutate_programs(inst, "nop", "jmp")
machine = Machine()

for prog in programs:
    machine.reset()
    machine.read_program(prog)
    try:
        print(machine.run())
    except RuntimeError as err:
        pass

8


In [8]:
prog = load_program("day08_data.txt")
programs = mutate_programs(prog, "nop", "jmp")
machine = Machine()

for prog in programs:
    machine.reset()
    machine.read_program(prog)
    try:
        print(machine.run())
    except RuntimeError as err:
        pass

2477
